In [1]:
import tensorflow as tf

# List all physical devices
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print(f"Number of GPUs available: {len(gpus)}")
    for gpu in gpus:
        print(f"GPU Name: {gpu}")
        # Get detailed information about the GPU
        details = tf.config.experimental.get_device_details(gpu)
        for key, value in details.items():
            print(f"{key}: {value}")
else:
    print("No GPU found.")

import subprocess

def get_gpu_memory():
    try:
        # Run nvidia-smi to get GPU memory info
        result = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=memory.total', '--format=csv,nounits,noheader'],
            encoding='utf-8'
        )
        # Split the result into lines and convert to integer
        gpu_memory = result.strip().split('\n')
        gpu_memory = [int(mem) for mem in gpu_memory]
        return gpu_memory
    except Exception as e:
        print(f"Error: {e}")
        return None

gpu_memory = get_gpu_memory()
if gpu_memory:
    for i, mem in enumerate(gpu_memory):
        print(f"GPU {i} Total memory: {mem / 1024:.2f} GB")
else:
    print("No GPU found or nvidia-smi not available.")

Number of GPUs available: 1
GPU Name: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
device_name: NVIDIA GeForce RTX 4090
compute_capability: (8, 9)
GPU 0 Total memory: 23.99 GB


In [2]:
import os
import numpy as np
import soundfile as sf
from tqdm import tqdm
import random
from spafe.features.mfcc import mfcc
from spafe.features.gfcc import gfcc
from spafe.features.lfcc import lfcc
from spafe.features.lpc import lpc
from spafe.features.msrcc import msrcc
from spafe.features.ngcc import ngcc
from spafe.features.pncc import pncc
from spafe.features.psrcc import psrcc
from spafe.features.rplp import rplp

In [3]:
def extract_feature(audio_file, feature_func, feature_name):
    # Load audio file
    signal, sample_rate = sf.read(audio_file)

    # Extract feature
    feature = feature_func(signal, fs=sample_rate)

    # Save feature to numpy file
    np.save(f"{os.path.splitext(audio_file)[0]}_{feature_name}.npy", feature)

if __name__ == "__main__":
    audio_file = "'F:\Awais_data\Datasets\Halftruth\HAD_train\train\conbine\*.wav'"

    # Extract and save each feature
    features = {
        "gfcc": gfcc,
        "lfcc": lfcc,
        "lpc": lpc,
        "mfcc": mfcc,
        "msrcc": msrcc,
        "ngcc": ngcc,
        "pncc": pncc,
        "psrcc": psrcc,
        "rplp": rplp,
    }

In [ ]:
import os
import numpy as np
import soundfile as sf  # Assuming you've imported soundfile as sf
from tqdm import tqdm

# Function to extract features
def extract_feature(audio_file, feature_func, max_frames):
    if not os.path.exists(audio_file):
        print(f"File '{audio_file}' not found. Skipping...")
        return None
    
    # Load audio file to get sample rate
    signal, sample_rate = sf.read(audio_file)

    # Load audio file up to 4 seconds
    signal = signal[:int(4 * sample_rate)]

    # Extract feature
    feature = feature_func(signal, fs=sample_rate)

    # Pad or truncate feature to fixed length
    if feature.shape[0] < max_frames:
        feature = np.pad(feature, ((0, max_frames - feature.shape[0]), (0, 0)), mode='constant')
    elif feature.shape[0] > max_frames:
        feature = feature[:max_frames, :]

    return feature

# Function to read labels
def read_labels(labels_file):
    labels_dict = {}
    with open(labels_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            audio_name = parts[1]
            label = parts[5]  # Assuming the label is at the 5th index
            label = 1 if label == 'spoof' else 0
            labels_dict[audio_name] = label
    return labels_dict

if __name__ == "__main__":
    audio_path = "F:\\Awais_data\\Datasets\\ASV21\\ASVspoof2021_DF_eval\\flac"
    labels_file = "F:\\Awais_data\\Datasets\\ASV21\\ASVspoof2021_DF_eval\\ASVspoof2021.DF.cm.eval.trl.txt.txt"
    output_feature_file = "F:\\Awais_data\\Datasets\\ASV21\\Features\\DF\\DF_eval_mfcc_features.npy"
    output_labels_file = "F:\\Awais_data\\Datasets\\ASV21\\Features\\DF\\DF_eval_mfcc_labels.npy"
    max_frames = 40  # Maximum number of frames to consider

    # Read labels
    labels_dict = read_labels(labels_file)

    # Extract features and store labels
    features_list = []
    labels_list = []

    # Iterate over the labeled audio files
    for audio_name, label in tqdm(labels_dict.items(), desc="Extracting features"):
        audio_file = os.path.join(audio_path, audio_name + ".flac")
        
        # Check if file exists
        if not os.path.exists(audio_file):
            print(f"File '{audio_file}' not found. Skipping...")
            continue
        
        # Extract feature
        feature = extract_feature(audio_file, mfcc, max_frames)
        
        # Store feature and label if feature extraction was successful
        if feature is not None:
            features_list.append(feature)
            labels_list.append(label)

    # Convert lists to numpy arrays
    features_array = np.array(features_list)
    labels_array = np.array(labels_list)

    # Save features and labels to numpy files
    np.save(output_feature_file, features_array)
    np.save(output_labels_file, labels_array)

Extracting features: 100%|█████████████████████████████████████████████████▊| 609161/611829 [20:20:41<10:04,  4.42it/s]